In [ ]:
import pandas as pd
import numpy as np
import config as cf
import matplotlib.pyplot as plt
import talib

In [ ]:
from datetime import datetime

### Loading data

In [ ]:
%store -r tickers

In [ ]:
tickers

In [ ]:
dfs = dict()
print("...loading...")
for ticker in tickers:
    symbol = ticker["symbol"]
    if str(symbol).endswith("USDT"):
        print(symbol, end=" ")
        filename = f"{cf.AGGREGATED_DATA_PATH}futures/{symbol}-4h.h5"
        df = pd.read_hdf(filename)
        dfs[symbol] = df

In [ ]:
len(dfs)

#### Get all coins key info

In [ ]:
len_list = []
coverage_list = []
last_list = []
TRADE_TIMEFRAME_IN_MS = 4 * 60 * 60 * 1000
for symbol in dfs.keys():
    len_list.append(len(dfs[symbol]))
    must_length = (dfs[symbol].index.max()-dfs[symbol].index.min())/TRADE_TIMEFRAME_IN_MS+1
    real_length = len(dfs[symbol])
    coverage_list.append(real_length/must_length)
    last_list.append(dfs[symbol].index.max())
stat_df = pd.DataFrame({"symbol": dfs.keys(), "length": len_list, "coverage":coverage_list, "last":last_list})

#### Test correlation between Bitcoin and alt coins

In [ ]:
macd_multiple = 2
macd_fast = 26
macd_slow=12
macd_signal=9
short_multiples = 1
medium_multiples = 3
long_multiples = 10

In [ ]:
selected_symbols = stat_df[(stat_df["coverage"]==1)&\
    (stat_df["length"]>600*6)]["symbol"].values.tolist()

In [ ]:
selected_symbols_include_btc = selected_symbols + ["BTCUSDT"]

In [ ]:
for symbol in selected_symbols_include_btc:
    df = dfs[symbol]
    _,_,df[f"{symbol}_short_macd"] = talib.MACD(df["Close"],fastperiod=macd_fast,\
        slowperiod=macd_slow,signalperiod=macd_signal)
    df[f"{symbol}_short_macd"] = df[f"{symbol}_short_macd"]/df["Close"]
    _,_,df[f"{symbol}_medium_macd"] = talib.MACD(df["Close"],fastperiod=macd_fast*medium_multiples,\
        slowperiod=macd_slow*medium_multiples,signalperiod=macd_signal*medium_multiples)
    df[f"{symbol}_medium_macd"] = df[f"{symbol}_medium_macd"]/df["Close"]
    _,_,df[f"{symbol}_long_macd"] = talib.MACD(df["Close"],fastperiod=macd_fast*long_multiples,\
        slowperiod=macd_slow*long_multiples,signalperiod=macd_signal*long_multiples)
    df[f"{symbol}_long_macd"] = df[f"{symbol}_long_macd"]/df["Close"]

df = pd.concat([dfs[symbol][[f"{symbol}_short_macd",f"{symbol}_medium_macd",f"{symbol}_long_macd"]] \
    for symbol in selected_symbols_include_btc],axis=1)
df.dropna(inplace=True)

In [ ]:
for symbol in selected_symbols:
    df[f"{symbol}_diff_short"] = df[f"{symbol}_short_macd"] - df["BTCUSDT_short_macd"]
    df[f"{symbol}_diff_medium"] = df[f"{symbol}_medium_macd"] - df["BTCUSDT_medium_macd"]
    df[f"{symbol}_diff_long"] = df[f"{symbol}_long_macd"] - df["BTCUSDT_long_macd"]

    #Version 1: calculating bull or bear based on short,medium, long time frame
    df[f"{symbol}_diff_short_bear"] = df[df["BTCUSDT_short_macd"]<0][f"{symbol}_short_macd"] - df[df["BTCUSDT_short_macd"]<0]["BTCUSDT_short_macd"]
    df[f"{symbol}_diff_short_bull"] = df[df["BTCUSDT_short_macd"]>0][f"{symbol}_short_macd"] - df[df["BTCUSDT_short_macd"]>0]["BTCUSDT_short_macd"]
    df[f"{symbol}_diff_medium_bear"] = df[df["BTCUSDT_medium_macd"]<0][f"{symbol}_medium_macd"] - df[df["BTCUSDT_medium_macd"]<0]["BTCUSDT_medium_macd"]
    df[f"{symbol}_diff_medium_bull"] = df[df["BTCUSDT_medium_macd"]>0][f"{symbol}_medium_macd"] - df[df["BTCUSDT_medium_macd"]>0]["BTCUSDT_medium_macd"]
    df[f"{symbol}_diff_long_bear"] = df[df["BTCUSDT_long_macd"]<0][f"{symbol}_long_macd"] - df[df["BTCUSDT_long_macd"]<0]["BTCUSDT_long_macd"]
    df[f"{symbol}_diff_long_bull"] = df[df["BTCUSDT_long_macd"]>0][f"{symbol}_long_macd"] - df[df["BTCUSDT_long_macd"]>0]["BTCUSDT_long_macd"]
    
    #Version 2: calculating bull or bear only based on long timeframe
    df[f"{symbol}_diff_short_bear_l"] = df[df["BTCUSDT_long_macd"]<0][f"{symbol}_short_macd"] - df[df["BTCUSDT_long_macd"]<0]["BTCUSDT_short_macd"]
    df[f"{symbol}_diff_short_bull_l"] = df[df["BTCUSDT_long_macd"]>0][f"{symbol}_short_macd"] - df[df["BTCUSDT_long_macd"]>0]["BTCUSDT_short_macd"]
    df[f"{symbol}_diff_medium_bear_l"] = df[df["BTCUSDT_long_macd"]<0][f"{symbol}_medium_macd"] - df[df["BTCUSDT_long_macd"]<0]["BTCUSDT_medium_macd"]
    df[f"{symbol}_diff_medium_bull_l"] = df[df["BTCUSDT_long_macd"]>0][f"{symbol}_medium_macd"] - df[df["BTCUSDT_long_macd"]>0]["BTCUSDT_medium_macd"]
    df[f"{symbol}_diff_long_bear_l"] = df[df["BTCUSDT_long_macd"]<0][f"{symbol}_long_macd"] - df[df["BTCUSDT_long_macd"]<0]["BTCUSDT_long_macd"]
    df[f"{symbol}_diff_long_bull_l"] = df[df["BTCUSDT_long_macd"]>0][f"{symbol}_long_macd"] - df[df["BTCUSDT_long_macd"]>0]["BTCUSDT_long_macd"]

In [ ]:
diff_short_mean = []
diff_medium_mean = []
diff_long_mean = []

diff_short_mean_bear = []
diff_medium_mean_bear = []
diff_long_mean_bear = []

diff_short_mean_bull = []
diff_medium_mean_bull = []
diff_long_mean_bull = []

diff_short_mean_square = []
diff_medium_mean_square = []
diff_long_mean_square = []

diff_short_mean_bull_l = []
diff_medium_mean_bull_l = []
diff_long_mean_bull_l = []

diff_short_mean_bear_l = []
diff_medium_mean_bear_l = []
diff_long_mean_bear_l = []

for symbol in selected_symbols:
    diff_short_mean.append(df[f"{symbol}_diff_short"].mean())
    diff_medium_mean.append(df[f"{symbol}_diff_medium"].mean())
    diff_long_mean.append(df[f"{symbol}_diff_long"].mean())
    
    diff_short_mean_bear.append(df[f"{symbol}_diff_short_bear"].mean())
    diff_medium_mean_bear.append(df[f"{symbol}_diff_medium_bear"].mean())
    diff_long_mean_bear.append(df[f"{symbol}_diff_long_bear"].mean())
    
    diff_short_mean_bull.append(df[f"{symbol}_diff_short_bull"].mean())
    diff_medium_mean_bull.append(df[f"{symbol}_diff_medium_bull"].mean())
    diff_long_mean_bull.append(df[f"{symbol}_diff_long_bull"].mean())
    
    diff_short_mean_bear_l.append(df[f"{symbol}_diff_short_bear_l"].mean())
    diff_medium_mean_bear_l.append(df[f"{symbol}_diff_medium_bear_l"].mean())
    diff_long_mean_bear_l.append(df[f"{symbol}_diff_long_bear_l"].mean())
    
    diff_short_mean_bull_l.append(df[f"{symbol}_diff_short_bull_l"].mean())
    diff_medium_mean_bull_l.append(df[f"{symbol}_diff_medium_bull_l"].mean())
    diff_long_mean_bull_l.append(df[f"{symbol}_diff_long_bull_l"].mean())
    
    diff_short_mean_square.append(np.sqrt(np.dot(df[f"{symbol}_diff_short"],df[f"{symbol}_diff_short"]))/len(df))
    diff_medium_mean_square.append(np.sqrt(np.dot(df[f"{symbol}_diff_medium"],df[f"{symbol}_diff_medium"]))/len(df))
    diff_long_mean_square.append(np.sqrt(np.dot(df[f"{symbol}_diff_long"],df[f"{symbol}_diff_long"]))/len(df))

In [ ]:
diff_df = pd.DataFrame({
    "diff_short_mean": diff_short_mean,
    "diff_medium_mean": diff_medium_mean,
    "diff_long_mean": diff_long_mean,
    "diff_short_mean_bear": diff_short_mean_bear,
    "diff_medium_mean_bear": diff_medium_mean_bear,
    "diff_long_mean_bear": diff_long_mean_bear,
    "diff_short_mean_bull": diff_short_mean_bull,
    "diff_medium_mean_bull": diff_medium_mean_bull,
    "diff_long_mean_bull": diff_long_mean_bull,
    "diff_short_mean_bear_l": diff_short_mean_bear_l,
    "diff_medium_mean_bear_l": diff_medium_mean_bear_l,
    "diff_long_mean_bear_l": diff_long_mean_bear_l,
    "diff_short_mean_bull_l": diff_short_mean_bull_l,
    "diff_medium_mean_bull_l": diff_medium_mean_bull_l,
    "diff_long_mean_bull_l": diff_long_mean_bull_l,
    "diff_short_mean_square":diff_short_mean_square,
    "diff_medium_mean_square":diff_medium_mean_square,
    "diff_long_mean_square":diff_long_mean_square})
diff_df.index = selected_symbols

##### Altcoin that has the highest and lowest correlation with bitcoin

In [ ]:
print(diff_df["diff_long_mean_square"].nlargest(5))
print(diff_df["diff_long_mean_square"].nsmallest(5))
print(diff_df["diff_medium_mean_square"].nlargest(5))
print(diff_df["diff_medium_mean_square"].nsmallest(5))
print(diff_df["diff_short_mean_square"].nlargest(5))
print(diff_df["diff_short_mean_square"].nsmallest(5))

##### Altcoin that drop slower than Bitcoin in bear market

In [ ]:
print(diff_df["diff_short_mean_bear"].nlargest(5))
print(diff_df["diff_medium_mean_bear"].nlargest(5))
print(diff_df["diff_long_mean_bear"].nlargest(5))

##### Altcoin that drop slower than Bitcoin in bear market (version 2)

In [ ]:
print(diff_df["diff_short_mean_bear_l"].nlargest(5))
print(diff_df["diff_medium_mean_bear_l"].nlargest(5))
print(diff_df["diff_long_mean_bear_l"].nlargest(5))

##### Altcoin that drop faster than Bitcoin in bear market

In [ ]:
print(diff_df["diff_short_mean_bear"].nsmallest(5))
print(diff_df["diff_medium_mean_bear"].nsmallest(5))
print(diff_df["diff_long_mean_bear"].nsmallest(5))

##### Altcoin that drop faster than Bitcoin in bear market (Version 2)

In [ ]:
print(diff_df["diff_short_mean_bear_l"].nsmallest(10))
print(diff_df["diff_medium_mean_bear_l"].nsmallest(10))
print(diff_df["diff_long_mean_bear_l"].nsmallest(10))

##### Altcoin that rise slower than Bitcoin in bull market

In [ ]:
print(diff_df["diff_short_mean_bull"].nsmallest(5))
print(diff_df["diff_medium_mean_bull"].nsmallest(5))
print(diff_df["diff_long_mean_bull"].nsmallest(5))

##### Altcoin that rise slower than Bitcoin in bull market (Version 2)

In [ ]:
print(diff_df["diff_short_mean_bull_l"].nsmallest(5))
print(diff_df["diff_medium_mean_bull_l"].nsmallest(5))
print(diff_df["diff_long_mean_bull_l"].nsmallest(5))

##### Altcoin that rise faster than Bitcoin in bull market

In [ ]:
print(diff_df["diff_short_mean_bull"].nlargest(5))
print(diff_df["diff_medium_mean_bull"].nlargest(5))
print(diff_df["diff_long_mean_bull"].nlargest(5))

##### Altcoin that rise faster than Bitcoin in bull market (Version 2)

In [ ]:
print(diff_df["diff_short_mean_bull_l"].nlargest(5))
print(diff_df["diff_medium_mean_bull_l"].nlargest(5))
print(diff_df["diff_long_mean_bull_l"].nlargest(5))